<a href="https://colab.research.google.com/github/shantanu812/MLProjects/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

assert sys.version_info >= (3, 7)

In [2]:
IS_COLAB = "google.colab" in sys.modules
if IS_COLAB:
    import os
    os.environ["TF_USE_LEGACY_KERAS"] = "1"
    import tf_keras

In [3]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [4]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

In [9]:
import tensorflow_datasets as tfds
raw_train_set,raw_valid_set,raw_test_set = tfds.load(name='imdb_reviews',split=['train[:90%]','train[90%:]','test'],as_supervised=True)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000,seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [12]:
for review,label in raw_train_set.take(4):
  print(review.numpy()[:200],'...')
  print('Label: ',label.numpy())

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting " ...
Label:  0
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However ' ...
Label:  0
b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun' ...
Label:  0
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf' ...
Label:  1


In [13]:
vocab_size=1000
text_vec_layer=tf.keras.layers.TextVectorization(max_tokens=vocab_size,output_mode='int',standardize='lower_and_strip_punctuation',split='whitespace')
text_vec_layer.adapt(train_set.map(lambda review,label:review))

In [14]:
embed_size=128
tf.random.set_seed(42)
model=tf.keras.Sequential([
  text_vec_layer,
  tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embed_size),
  tf.keras.layers.GRU(128),
  tf.keras.layers.Dense(1,activation='sigmoid')

  ])
model.compile(loss="binary_crossentropy", optimizer="nadam",
metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 49s 62ms/step - loss: 0.6935 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.5008
Epoch 2/2
704/704 [==============================] - 25s 36ms/step - loss: 0.6926 - accuracy: 0.5050 - val_loss: 0.6939 - val_accuracy: 0.4988


In [15]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=7)

Epoch 1/7
704/704 [==============================] - 50s 64ms/step - loss: 0.4779 - accuracy: 0.7663 - val_loss: 0.4820 - val_accuracy: 0.7924
Epoch 2/7
704/704 [==============================] - 26s 37ms/step - loss: 0.3511 - accuracy: 0.8525 - val_loss: 0.3356 - val_accuracy: 0.8616
Epoch 3/7
704/704 [==============================] - 24s 34ms/step - loss: 0.3017 - accuracy: 0.8750 - val_loss: 0.3164 - val_accuracy: 0.8708
Epoch 4/7
704/704 [==============================] - 25s 35ms/step - loss: 0.2740 - accuracy: 0.8890 - val_loss: 0.3097 - val_accuracy: 0.8704
Epoch 5/7
704/704 [==============================] - 24s 33ms/step - loss: 0.2547 - accuracy: 0.8983 - val_loss: 0.3308 - val_accuracy: 0.8488
Epoch 6/7
704/704 [==============================] - 23s 33ms/step - loss: 0.2415 - accuracy: 0.9025 - val_loss: 0.3596 - val_accuracy: 0.8524
Epoch 7/7
704/704 [==============================] - 23s 33ms/step - loss: 0.2229 - accuracy: 0.9123 - val_loss: 0.3356 - val_accuracy: 0.8604

In [17]:
#prediction
test_loss,test_acc=model.evaluate(test_set)
print(test_loss,test_acc)

782/782 [==============================] - 30s 38ms/step - loss: 0.3882 - accuracy: 0.8238
0.3881760239601135 0.8238400220870972


In [18]:
new_review = "This movie was fantastic! I loved every minute of it."
print(f"Review: {new_review}")
print(f"Prediction: {model.predict([new_review])[0][0]:.2f}")

Review: This movie was fantastic! I loved every minute of it.
1/1 [==============================] - 0s 412ms/step
Prediction: 0.94


In [19]:
difficult_review = "This film wasn't bad, but it certainly wasn't great either. The plot was a bit convoluted, and the acting was mediocre. I wouldn't recommend it, but I wouldn't tell you to avoid it either."
print(f"Review: {difficult_review}")
print(f"Prediction: {model.predict([difficult_review])[0][0]:.2f}")

Review: This film wasn't bad, but it certainly wasn't great either. The plot was a bit convoluted, and the acting was mediocre. I wouldn't recommend it, but I wouldn't tell you to avoid it either.
1/1 [==============================] - 0s 35ms/step
Prediction: 0.05


In [ ]:
model.save("my_sent_analysis.keras")